In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 20)

We first import from data.un.org the statistics of population in urban areas.
If the link does not work anymore, feel free to use ./data/SYB61_253
We rename the first column to Region or Country

In [ ]:
statistic_df = pd.read_csv(
    'https://data.un.org/_Docs/SYB/CSV/SYB61_253_Population%20Growth%20Rates%20in%20Urban%20areas%20and%20Capital%20cities.csv',
    encoding='latin-1',
    header=1,
    usecols=range(1,9)
)
statistic_df.rename(columns = {
    statistic_df.columns[0]: 'Region or Country', 'Value': 'Urban Percentage'
}, inplace=True)

After the import is done data cleaning operation are done.
1. Filter out on Urban population (percent)
2. For each Region or Country get the data for 2018

In [ ]:
series_filter = statistic_df['Series'] == 'Urban population (percent)'
year_filter = statistic_df['Year'] == 2018
urban_2018_df = statistic_df[series_filter & year_filter]
urban_2018_df = urban_2018_df[['Region or Country', 'Urban Percentage']]
urban_2018_df = urban_2018_df.astype({'Urban Percentage': 'float'})

Before drawing the map we need to transform our pandas dataframe in a geopandas dataframe, in order to assign the positions on the map.

In [ ]:
import geopandas
country_geopandas = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
                                                                         

Before merging the urban data with geopandas let's make a check on country names between geopandas and urban_2018_df.


In [ ]:
country_geopandas_country_list = country_geopandas['name'].tolist()

In [ ]:
urban_2018_country_list = urban_2018_df['Region or Country'].tolist()


In [ ]:
set(country_geopandas_country_list) - set(urban_2018_country_list)

In [ ]:
countries_to_correct = [
    {'df_country_name': 'Czech Republic' , 'geo_country_name': 'Czechia'}
]

for country_to_correct in countries_to_correct:
    filter = urban_2018_df['Region or Country'] == country_to_correct['df_country_name']
    urban_2018_df.loc[filter, 'Region or Country'] = country_to_correct['geo_country_name']
urban_2018_country_list = urban_2018_df['Region or Country'].tolist()

In [ ]:
country_geopandas = country_geopandas.merge(
    urban_2018_df, 
    how='inner', left_on=['name'], right_on=['Region or Country']
)

In this tutorial we will be using folium to draw the map. Read all about it here: https://python-visualization.github.io/folium/
Let's check first the overlap on country names with folium

In [ ]:
import json
import urllib.request
import folium
url = 'https://raw.githubusercontent.com/python-visualization/folium/main/examples/data/world-countries.json'
data = urllib.request.urlopen(url).read()
folium_country_json = json.loads(data)
folium_country_list = [rec['properties']['name'] for rec in folium_country_json['features']]

In [ ]:
country_geopandas_country_list = country_geopandas['name'].tolist()

In [ ]:
len(set(country_geopandas_country_list) & set(folium_country_list))

In folium data we have more than 170 countries. There seems to be a mismatch on more than 30 countries across the 3 data sources. You can clean the data as done above.
Now on the drawing part!


You can find here the variations of colors: https://github.com/python-visualization/branca/blob/master/branca/scheme_base_codes.json

Now that we cleaned the data (more or less, I leave up to you to refine it further, there are 17 countries missing) we can plot it and draw the map!
First we need to make sure that we have in the urban_2018_df only the matching countries.


In [ ]:
urban_area_map = folium.Map()
folium.Choropleth(
    geo_data=country_geopandas,
    name='choropleth',
    data=country_geopandas,
    columns=['Region or Country', 'Urban Percentage'],
    key_on='feature.properties.name',
    fill_color='Greens',
    nan_fill_color='Grey',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Percentage of population living in Urban areas'
).add_to(urban_area_map)

In [ ]:
urban_area_map.save('urban_population_2018.html')